In [ ]:
import numpy as np
import pandas as pd
import zipfile
import os
import urllib.request
from tqdm import tqdm
from pathlib import Path
import cufflinks
cufflinks.go_offline()

In [ ]:
summary = pd.read_excel("data/Statystyki_2000-2018_wer20180828.xlsx", sheet_name="PM2,5")
summary = summary[summary['Czas uśredniania'] == '1g']

In [ ]:
summary.columns

In [ ]:
stations = summary['Kod stacji'][summary['Kod stacji'].str.startswith("")].unique()

In [ ]:
val_key = "Średnia"
summary[summary['Kod stacji'].isin(stations)].pivot(index="Rok", columns="Kod stacji", values=val_key).iplot()

In [ ]:
val_key = "Maks"
summary[summary['Kod stacji'].isin(stations)].pivot(index="Rok", columns="Kod stacji", values=val_key).iplot()

Specify Urls with the zipped sensor data files

In [ ]:
urls={
    '2000': 'http://powietrze.gios.gov.pl/pjp/archives/downloadFile/223',
    '2001': 'http://powietrze.gios.gov.pl/pjp/archives/downloadFile/224',
    '2002': 'http://powietrze.gios.gov.pl/pjp/archives/downloadFile/225',
    '2003': 'http://powietrze.gios.gov.pl/pjp/archives/downloadFile/226',
    '2004': 'http://powietrze.gios.gov.pl/pjp/archives/downloadFile/202',
    '2005': 'http://powietrze.gios.gov.pl/pjp/archives/downloadFile/203',
    '2006': 'http://powietrze.gios.gov.pl/pjp/archives/downloadFile/227',
    '2007': 'http://powietrze.gios.gov.pl/pjp/archives/downloadFile/228',
    '2008': 'http://powietrze.gios.gov.pl/pjp/archives/downloadFile/229',
    '2009': 'http://powietrze.gios.gov.pl/pjp/archives/downloadFile/230',
    '2010': 'http://powietrze.gios.gov.pl/pjp/archives/downloadFile/231',
    '2011': 'http://powietrze.gios.gov.pl/pjp/archives/downloadFile/232',
    '2012': 'http://powietrze.gios.gov.pl/pjp/archives/downloadFile/233',
    '2013': 'http://powietrze.gios.gov.pl/pjp/archives/downloadFile/234',
    '2014': 'http://powietrze.gios.gov.pl/pjp/archives/downloadFile/235',
    '2015': 'http://powietrze.gios.gov.pl/pjp/archives/downloadFile/236',
    '2016': 'http://powietrze.gios.gov.pl/pjp/archives/downloadFile/242',
    '2017': 'http://powietrze.gios.gov.pl/pjp/archives/downloadFile/262',
}
zip_dir = Path('./data/zip')
extracted_dir = Path("data/extracted/")

Download the files

In [ ]:
for url in tqdm(urls.values()):
    name = zip_dir/url.split('/')[-1]
    urllib.request.urlretrieve(url, name)
    zipfile.ZipFile(name, 'r').extractall(extracted_dir)

Download the metadata file

In [ ]:
def merc_from_arrays(ϕ, λ):
    ϕ = ϕ / (180/np.pi)
    λ = λ / (180/np.pi)
    r_major = 6378137.000
    x = r_major * λ
    y = r_major * np.log(np.tan(np.pi/4+ϕ/2))
    return (x,y)

def add_web_mercator(metadata):
    E, N = merc_from_arrays(metadata['WGS84 φ N'], metadata['WGS84 λ E'])
    metadata['web_mercator_E'] = E
    metadata['web_mercator_N'] = N

metadata_url = "http://powietrze.gios.gov.pl/pjp/archives/downloadFile/265"
metadata_file = extracted_dir / "metadata.xlsl"
urllib.request.urlretrieve(metadata_url, metadata_file)
metadata = pd.read_excel(metadata_file).drop(['Nr'], axis=1)

# clean up the -999.0 values for latitude and longitude
for col in ['WGS84 φ N','WGS84 λ E']:
    metadata.loc[metadata[col] < -360, col] = np.nan

add_web_mercator(metadata)

In [ ]:
import datetime
    
def detect_kod_stacji(df):
    for idx, row in df.reset_index(drop=True).head(10).iterrows():
        try:
            pd.to_numeric(row)
        except:
            if row.str.contains("^[A-Z][a-z]").mean()>0.9:
                return idx
    raise Exception(f"Could not find Kod stacji, {df.head(10)}")
    
def detect_values(df):
    return df.index.map(lambda x: isinstance(x,datetime.datetime))

def format_table(df):
    kod_stacji_idx = detect_kod_stacji(df)
    values_idx = detect_values(df)
    df_new = df[values_idx].copy()
    df_new.index = pd.to_datetime(df_new.index)
    df_new.index.names=["Date"]
    df_new.columns = list(df.iloc[kod_stacji_idx])
    return df_new

def convert_to_kod_stacji(df, metadata):
    if not set(df.columns).difference(metadata['Kod stacji']):
        return
    
    df.columns = list(pd.DataFrame(index=df.columns)
     .merge(
         metadata[['Kod stacji', 'Stary Kod stacji']],
         how='left',
         left_index=True,
         right_on="Stary Kod stacji")
     .loc[:,'Kod stacji']
    )
    
def get_file_name(year, measure, interval=24):
    return "_".join([str(year), measure, str(interval)+"g"+".xlsx"])

def convert_to_number(x):
    if type(x) is str:
        return float(x.replace(",","."))
    else:
        return x

def to_dataframe(xls, metadata):
    
    if not xls.exists():
        xls = Path(str(xls).replace("PM2.5", "PM25"))
    
    if not xls.exists():
        return pd.DataFrame()
    
    df = pd.read_excel(xls, header=None, index_col=0)
    df = format_table(df)
    convert_to_kod_stacji(df, metadata)
    df = df.applymap(convert_to_number)
    df = df.loc[:,~df.columns.duplicated()]
    df = df.astype(np.float32)
    df.drop(df.columns[df.columns.isnull()], axis=1, inplace=True)
    return df

In [ ]:
for indicator in tqdm(["PM2.5", "PM10", "NO2", "SO2"]):
    lst = {year : to_dataframe(extracted_dir / get_file_name(year, indicator, interval=24), metadata)
               for year in urls.keys()}
    all_data = pd.concat(lst.values(), axis=0, sort=False)
    all_data.to_hdf('data/data.h5', f"24g/{indicator}")
metadata.to_hdf('data/data.h5', "metadata")

In [ ]:
all_data = pd.read_hdf('data/data.h5', "24g/PM10")

In [ ]:
((all_data>50).groupby(pd.Grouper(freq="Y")).sum()>35).sum(0).sort_values().head(10)

In [ ]:
cols = all_data.columns.str.startswith("MpKrak")
all_data.rolling(30).mean().iloc[:,cols].iplot()

In [ ]:
all_data.iloc[:,cols].groupby(pd.Grouper(freq="M")).median().iplot()

In [ ]:
import pandas_bokeh
pandas_bokeh.output_notebook()

In [ ]:
E_minmax = metadata['web_mercator_E'].min(), metadata['web_mercator_E'].max()
N_minmax = metadata['web_mercator_N'].min(), metadata['web_mercator_N'].max()

In [ ]:
summary_stats.index

In [ ]:
summary_stats = all_data.groupby(pd.Grouper(freq="M")).mean()
dates = summary_stats.index #.strftime('%Y-%M').values
data = {}
for date in dates:
    key = date.strftime("%Y-%M")
    df = metadata.copy().set_index('Kod stacji')
    df['summary'] = summary_stats.loc[dates[0]].T
    data[key] = ColumnDataSource(df)

In [ ]:
from bokeh.layouts import row, column, grid
from bokeh.io import output_file, show
from bokeh.plotting import figure
from bokeh.tile_providers import get_provider
from bokeh.models import HoverTool, PanTool, WheelZoomTool, DateRangeSlider
from bokeh.models import ColumnDataSource

source = ColumnDataSource(metadata)

# def slider_update(attrname, old, new):
#     year = slider.value
#     label.text = str(year)
#     source.data = data[year]

# slider = DateRangeSlider(start=dates[0].date(), end=dates[-1].date(), title="Year-Month")
# slider.on_change('value', slider_update)

hover = HoverTool(
        tooltips=[
            ("Stacja", "@{Nazwa stacji}")
        ]
    )

fig = figure(
    tools=[hover, PanTool(), WheelZoomTool()],
    x_range=E_minmax,
    y_range=N_minmax,
    x_axis_type="mercator",
    y_axis_type="mercator")
fig.add_tile(get_provider('CARTODBPOSITRON_RETINA'))
fig.circle(
    x='web_mercator_E',
    y='web_mercator_N',
    source=list(data.values())[0],
    line_color='grey',
    fill_color='yellow')

show(fig)

# # Set up layouts and add to document
# inputs = column(slider)

# show(row(inputs, fig, width=800))